In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/CB.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MACC","6MACC","9MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsshot.csv', index=False)


Epoch 1/100
264/264 [==============================] - 1s 2ms/step - loss: 6.5795 - mean_absolute_error: 1.8411 - val_loss: 4.9500 - val_mean_absolute_error: 1.6719
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.0619 - mean_absolute_error: 1.5646 - val_loss: 3.9999 - val_mean_absolute_error: 1.5495
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5660 - mean_absolute_error: 1.4838 - val_loss: 3.6533 - val_mean_absolute_error: 1.4981
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.3348 - mean_absolute_error: 1.4444 - val_loss: 3.4435 - val_mean_absolute_error: 1.4510
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.1694 - mean_absolute_error: 1.4096 - val_loss: 3.2820 - val_mean_absolute_error: 1.4170
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.0308 - mean_absolute_error: 1.3772 - val_loss: 3.1497 - val_mean_absolute_error: 1.3798
Epoch 7/10

C:\Users\Wandi\AppData\Local\Temp\ipykernel_140556\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 4.8453 - mean_absolute_error: 1.8302 - val_loss: 4.7712 - val_mean_absolute_error: 1.7381
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.2098 - mean_absolute_error: 1.6387 - val_loss: 4.2754 - val_mean_absolute_error: 1.6256
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.8293 - mean_absolute_error: 1.5557 - val_loss: 3.9357 - val_mean_absolute_error: 1.5642
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5948 - mean_absolute_error: 1.5104 - val_loss: 3.7320 - val_mean_absolute_error: 1.5185
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4195 - mean_absolute_error: 1.4711 - val_loss: 3.5591 - val_mean_absolute_error: 1.4860
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.2648 - mean_absolute_error: 1.4378 - val_loss: 3.3977 - val_mean_absolute_error: 1.4452
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_140556\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 5.0408 - mean_absolute_error: 1.8786 - val_loss: 4.9112 - val_mean_absolute_error: 1.7930
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.3903 - mean_absolute_error: 1.6901 - val_loss: 4.4920 - val_mean_absolute_error: 1.6601
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 4.0452 - mean_absolute_error: 1.5997 - val_loss: 4.1681 - val_mean_absolute_error: 1.6158
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.7920 - mean_absolute_error: 1.5570 - val_loss: 3.9437 - val_mean_absolute_error: 1.5682
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5960 - mean_absolute_error: 1.5204 - val_loss: 3.7511 - val_mean_absolute_error: 1.5173
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4133 - mean_absolute_error: 1.4713 - val_loss: 3.5585 - val_mean_absolute_error: 1.5054
Epoch 7/100
264/264 [=

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_140556\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 6.9495 - mean_absolute_error: 1.9219 - val_loss: 4.8853 - val_mean_absolute_error: 1.6485
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.1336 - mean_absolute_error: 1.5782 - val_loss: 4.1028 - val_mean_absolute_error: 1.5844
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.7255 - mean_absolute_error: 1.5280 - val_loss: 3.8254 - val_mean_absolute_error: 1.5251
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4973 - mean_absolute_error: 1.4812 - val_loss: 3.6265 - val_mean_absolute_error: 1.4848
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.3216 - mean_absolute_error: 1.4423 - val_loss: 3.4720 - val_mean_absolute_error: 1.4395
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.1788 - mean_absolute_error: 1.4068 - val_loss: 3.3077 - val_mean_absolute_error: 1.4229
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_140556\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


264/264 [==============================] - 1s 2ms/step - loss: 6.5566 - mean_absolute_error: 1.8616 - val_loss: 4.7702 - val_mean_absolute_error: 1.6326
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 4.1705 - mean_absolute_error: 1.5874 - val_loss: 4.2375 - val_mean_absolute_error: 1.6056
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 3.8963 - mean_absolute_error: 1.5584 - val_loss: 4.0434 - val_mean_absolute_error: 1.5689
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 3.7251 - mean_absolute_error: 1.5251 - val_loss: 3.8842 - val_mean_absolute_error: 1.5308
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 3.5763 - mean_absolute_error: 1.4912 - val_loss: 3.7342 - val_mean_absolute_error: 1.5004
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 3.4302 - mean_absolute_error: 1.4621 - val_loss: 3.5906 - val_mean_absolute_error: 1.4664
Epoch 7/100
264/264 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_140556\1078879232.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
